## extract specific classes with images

In [1]:
import numpy as np
import cv2
import os
import glob

folder_path = '/dataset/Dataset_backup/Completion_dataset/basics_detector/5class/2021_11_03'
folders = os.listdir(folder_path)

count = 0
count2 = 0
count3 = 0
target_class = '0'
target_class2 = '1'
target_class3 = '2'
limit = 0
error = 0
for folder in folders:
    path = os.path.join(folder_path, folder)
    image_files = glob.glob(f'{path}/*.jpg') + glob.glob(f'{path}/*.png')
    for filename in image_files:
        label = filename[:filename.rfind('.')] + '.txt'
        try:
            f = open(label, 'r')
            line = f.readlines()
            last_count = count
            last_count2 = count2
            last_count3 = count3
            if(len(line) > 1):
                for i in line:
                    if(i[0] != target_class):
                        #break ##?
                        continue ##?
                    
                img_path = path[:path.find('2021_11_03')]+'2024_02_06/person/' + filename[filename.rfind('/')+1:]
                txt_path = img_path[:img_path.find('.')] + '.txt'
                txt = open(txt_path, "w")
                
                img = cv2.imread(filename)
                cv2.imwrite(img_path, img)
                
                for i in line:
                    if(i[0] == target_class):
                        count += 1        
                        txt.write(target_class+i[i.find(' '):]) ## if you want replace class number
                    elif(i[0] == target_class2):
                        count2 += 1
                        txt.write(target_class2+i[i.find(' '):])
                    elif(i[0] == target_class3):
                        count3 += 1
                        txt.write(target_class3+i[i.find(' '):])

                nofound = 1
                if(last_count==count):
                    nofound = 1
                elif(last_count2==count2):
                    nofound = 1
                elif(last_count3==count3):
                    nofound = 1
                else:
                    nofound = 0
                
                if(nofound):
                    os.remove(txt_path)
                    os.remove(img_path)
                    
                txt.close()
            f.close()
        except:
            error += 1
    #     limit += 1
    #     if(limit == 10000):
    #         break
    # break

print("count: %d\n",count)
print("count2: %d\n",count2)
print("count3: %d\n",count3)
print("error: %d\n",error)

count: %d
 779745
count: %d
 127331
count: %d
 469712
error: %d
 0
